In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
 
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

In [ ]:
#Apply model to crawler data
import pandas as pd
import numpy as np

file_path = '/Users/Downloads/summer term/covid/Vomit/202212.csv'
label_column_name='Embedded_text'
df = pd.read_csv(file_path)

    #Get the value of label column
labels = df[label_column_name]

df['PV'] = ''
df['score'] = ''

text = ''
texts = ''
count = 0
max_score = 0
for label in labels:
    if True:
        text=f"text='{label}' \n"
        #print (text)
        texts = text.replace(' \n',' ')
        #print (texts)
        text = preprocess(text)
        #print (text)
        #print('--------------------------------------------------------------------------------------------------------')
        
        #tokenizer = tokenizer.encode(texts,)
        #encoded_input = tokenizer(texts, pad_to_max_length=True, return_tensors='pt')
        encoded_input = tokenizer(texts, return_tensors='pt')
        try:
            output = (**encoded_input)
        except ValueError:
            print('skip')
            
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        print(scores)
        print(scores)
        PV = scores
        df.at[count, 'PV'] = scores
        max_score = max(scores)  #Get the maximum value in the score
        max_index = np.argmax(scores)   #Get the index of the maximum value
        
        print("max_score")
        print(max_score)
        
        print("max_index")
        print(max_index)
        
        print("df.at[count, 'sentiment']")
        if max_index == 0:
            df.at[count, 'sentiment'] =  float(max_score) #Negative - we focus on
        else 
            df.at[count, 'sentiment'] =  float(0) #Neutral(index = 1) & Positive(index = 2)
        print(df.at[count, 'sentiment'])
        print('-----------------------------------------------------------------------------------------------------------------')
        